## Análise de Negócios
### Analise dos dados de preço do petróleo brent

<img src = 'preco_Petroleo.png'>

### Fomos contratado para uma consultoria, e nosso trabalho foi analisar os dados de preço do petróleo brent, para construir um dashboard.

<br/>
A solicitação do nosso cliente, foi que desenvolvessemos um dashboard interativo e que gere insights relevantes para tomada de decisão. <br/>

<br/>
Além disso, solicitou que fosse desenvolvido um modelo de Machine Learning para fazer o forecasting do preço do petróleo.<br/>

1 - Trabalhamos com Dados Existentes no site do ipea <br/>
link: http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view <br/>


In [ ]:
"""
CREATE TABLE anp.preco_petroleo(
		 data_			date
		,Preco_			float
		
)
"""

'\nCREATE TABLE anp.preco_combustivel(\n\t\tregiao \t\t\t\tvarchar(255)\n\t\t,estado\t\t\t\tvarchar(255)\n\t\t,municipio\t\t\tvarchar(255)\n\t\t,revenda\t\t\tvarchar(255)\n\t\t,cnpj\t\t\t\tvarchar(255)\n\t\t,nome_rua\t\t\tvarchar(255)\t\n\t\t,numero_rua\t\t\tvarchar(255)\n\t\t,complemento\t\tvarchar(255)\n\t\t,bairro\t\t\t\tvarchar(255)\n\t\t,cep\t\t\t\tvarchar(255)\n\t\t,produto\t\t\tvarchar(255)\n\t\t,data_coleta\t\tdate\n\t\t,valor_venda\t\tfloat\n\t\t,unidade_medida\t\tvarchar(255)\n\t\t,bandeira\t\t\tvarchar(255)\n\n)\n'

<img src = 'PostgreSQL-logo.png'>
<img src = 'postgre1.png'>

3 - Utilizar esses dados para as nossas análises <br/>

Instalando as bibliotecas

In [15]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
pip install psycopg2


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import pandas as pd

C:\Users\Sergio\AppData\Local\Temp\ipykernel_14908\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [18]:
#Importando as Bibliotecas

import requests                   #Para acessar a URL
from bs4 import BeautifulSoup     #Para acessar o HTML
import pandas as pd

# URL do site IPEADATA
url = 'http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view'

# Faz uma requisicăo GET ao site e captura a resposta
response = requests.get(url)

# Verifica se a requisição foi bem sucedida
if response.status_code == 200:
    # Cria um objeto BeautifulSoup para analisar o HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Procura pela tabela no HTML analisado (o id ou classe pode variar)
    # Inspecionar o HTML para obter o seletor correto
    table = soup.find('table', {'id': 'grd_DXMainTable'})

    # Usa ○ pandas para ler a tabela HTML diretamente para um DataFrane
    df = pd.read_html(str(table),skiprows=0)[0]

    df.columns = df.iloc[0]
    df = df.drop(0)

   # Mostra as primeiras linhas do DataFrame
    df.head()
else:
      print('Falha ao acessar a página: Status Code', response.status_code)


C:\Users\Sergio\AppData\Local\Temp\ipykernel_14908\2000270153.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table),skiprows=0)[0]


1.1 - **Entendimento dos Dados:** 
Os preços de petróleo se estende de 1987 até 2024. É importante notar que esta série temporal é bastante longa, o que significa que ela pode conter várias tendências, sazonalidades e ciclos econômicos.

In [19]:
df

,Data,Preço - petróleo bruto - Brent (FOB)
1,12/02/2024,8388
2,09/02/2024,8358
3,08/02/2024,8301
4,07/02/2024,8118
5,06/02/2024,8046
...,...,...
11103,26/05/1987,1863
11104,25/05/1987,1860
11105,22/05/1987,1855
11106,21/05/1987,1845


In [37]:
# Renomear a coluna 'Preço - petróleo bruto - Brent (FOB)' para 'Preco'
df = df.rename(columns={'Preço - petróleo bruto - Brent (FOB)': 'preco'})
df = df.rename(columns={'Data': 'data'})

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11107 entries, 11107 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   data_   11107 non-null  datetime64[ns]
 1   preco_  11107 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 260.3+ KB


1.2 - **Pré-processamento dos Dados:** As datas devem ser tratadas como objetos de data/tempo e os preços como valores numéricos.

In [40]:
# Converte a coluna 'Data' para datetime

df['data_'] = pd.to_datetime(df['data_'], format='%d/%m/%Y')
# Ordenar o DataFrame pela coluna 'Data' de forma ascendente
df = df.sort_values(by='data_', ascending=True)

# Em seguida, divida a coluna 'Preço - petróleo bruto - Brent (FOB)' por 100
df['preco_'] = df['preco_'].astype(float).round(2)/ 100

In [41]:
import pandas.io.sql as sqlio

In [42]:
import psycopg2 as ps

In [43]:
conn = ps.connect(dbname='ANP',
                  user='postgres',
                  password='postgres',
                  host='localhost',
                  port='5432')


In [44]:
cursor = conn.cursor()

create_table_query = '''
CREATE TABLE IF NOT EXISTS preco_petroleo (
    Data DATE,
    Preco float
);
'''
cursor.execute(create_table_query)
conn.commit()


In [45]:
import io

# Copie o DataFrame para um buffer de texto
buffer = io.StringIO()
df.to_csv(buffer, index=False, header=False)
buffer.seek(0)

# Use a conexão para copiar os dados do buffer para a tabela
with conn.cursor() as cursor:
    cursor.copy_from(buffer, 'preco_petroleo', sep=',')
    conn.commit()


In [46]:
cursor.close()
conn.close()


In [47]:
df.head()

,data_,preco_
11107,1987-05-20,18.63
11106,1987-05-21,18.45
11105,1987-05-22,18.55
11104,1987-05-25,18.60
11103,1987-05-26,18.63


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11107 entries, 11107 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   data_   11107 non-null  datetime64[ns]
 1   preco_  11107 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 260.3 KB


In [49]:
df.shape

(11107, 2)